# CDD Streaming

In [ ]:
!pip3 install opencv-python  > /dev/null
!pip3 install albumentations > /dev/null

In [53]:
import cv2 as cv
import albumentations as a


def get_training_augmentation(_src, _bboxes, _category_id, _size):
    transform = a.Compose([a.Resize(_size[0], _size[1])], bbox_params=a.BboxParams(format='coco', label_fields=['category_id']))
    return transform(image=_src, bboxes=_bboxes, category_id=_category_id)['bboxes']


# BGR
BOX_COLOR = (0, 0, 225) # Red
BOX_THICKESS = 5
TEXT_COLOR = (255, 255, 255) # White
TEXT_SIZE = 0.9
TEXT_FONT = 0
IMG_SIZE = (200, 200)

category_id_to_name = { 1: "dent", 2: "broken_glass", 3: "deflated_wheel", 4: "scratch", 5: "broken_headlight"}

def show_webcam(mirror=False):
    cam = cv.VideoCapture(0)
    for fps in range(5000):
        _, img = cam.read()
        #img = cv.resize(img, (round(200+fps), round(200+fps)))
        size = img.shape

##########
###Back###
##########
        img_crop = cv.resize(img, IMG_SIZE)
        #########################################################
        # bbox, category_id = model.detect(img_crop) 
        bbox = [10, 10, 150, 150]
        category_id = [1]
        #########################################################
        transformed_bboxes = get_training_augmentation( img_crop,       \
                                                        [bbox],         \
                                                        [category_id],  \
                                                        size           \
                                                        )
##########
###Back###
##########

        if mirror: 
            img = cv.flip(img, 1)

        for left, top, right, bottom in transformed_bboxes:
            left, top, right, bottom = int(left), int(top), int(right), int(bottom)
            cv.rectangle(img,(left,top),(right, bottom), BOX_COLOR, BOX_THICKESS)
            cv.putText(img, category_id_to_name.get(category_id[0]), (right+10, bottom), TEXT_FONT, TEXT_SIZE, TEXT_COLOR)

        cv.imshow('my webcam', img)
        if cv.waitKey(1) == 27: 
            break  # esc to quit
    cv.destroyAllWindows()

show_webcam(mirror=True)

# Face Detection Streaming

In [1]:
!pip3 install dlib             > /dev/null
!pip3 install face_recognition > /dev/null
!pip3 install opencv-python    > /dev/null

In [55]:
# import libraries
import cv2 as cv
import face_recognition
 
video_capture = cv.VideoCapture(0)
face_locations = []
 
while True:
    ret, frame = video_capture.read()
    frame = cv.flip(frame, 1)      # mirror
    rgb_frame = frame[:, :, ::-1]  # to RGB

    face_locations = face_recognition.face_locations(rgb_frame)
    
    for top, right, bottom, left in face_locations:
        cv.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

    cv.imshow('Video', frame)      
    if cv.waitKey(1) == 27: 
        break  # esc to quit
 
video_capture.release()
cv.destroyAllWindows()